In [25]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
def get_dataset(data_dir):
    transform = transforms.Compose([
            transforms.ToTensor()
        ])
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True, download= True,  transform=transform,
    )
    test_dataset = datasets.CIFAR10(
        root=data_dir, train=False, download= True,  transform=transform,
    )
    full_dataset = ConcatDataset([train_dataset, test_dataset])
 
    train_loader = torch.utils.data.DataLoader(
    full_dataset, batch_size=1,
     num_workers=2
)




    return (full_dataset, test_dataset.classes)




data_loader, classes = get_dataset(data_dir = './data/cifar10_1batch')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
from PIL import Image
data_iter = iter(data_loader)
images, labels = next(data_iter)
# Iterate through the DataLoader
for batch in data_loader:
    # Assuming the batch contains 'image' and 'label' keys
    images = batch['image']
    labels = batch['label']

    # Iterate through the images in the batch
    for i in range(len(images)):
        image = images[i]
        label = labels[i]

        # Convert the tensor image to PIL Image
        image_pil = transforms.ToPILImage()(image)

        # Save the image
        image_pil.save(f'image_{i}.jpg')

NameError: name 'data_loader' is not defined